In [1]:
import PyPDF2
import pandas as pd
from unidecode import unidecode
import re

In [4]:
#folder_path = "C:/Users/User/Documents/GitHub/CNHGitHub/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"
folder_path = "D:/1. Documentos/0. Bases de datos/0. Dofiles y Scripts/CNH_rep/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"

def extraeTextoPdf(pdfPath, textIni, textFin):
    with open(pdfPath, 'rb') as pdfFile:
        # Create a PDF reader object
        pdfReader = PyPDF2.PdfReader(pdfFile)
    
        # Iterate through each page in the PDF
        contPagesIni = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textIni in testText:
                break
            else:
                contPagesIni += 1

        contPagesFin = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textFin in testText:
                break
            else:
                contPagesFin += 1
        
        # Initialize a variable to store the extracted text
        extracted_text1 = ''
        for pageNum in range(contPagesIni,contPagesFin +1):
            # Extract text from the current page
            page2 = pdfReader.pages[pageNum]
            extracted_text1 += page2.extract_text()

        patron = re.compile(f'{textIni}(.*?){textFin}', re.DOTALL)
    
        resultado = patron.search(extracted_text1)
        extracted_text = resultado.group(1)
    
    return extracted_text
       

In [5]:
# Open the PDF file in read-binary mode
pdfPath = folder_path + '234_202_3.pdf'

# En un solo departamento
conflictosActivos1 = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.1 CONFLICTOS ACTIVOS", textFin= "5.2 CONFLICTOS")

# En más de un departamento
conflictosActivos2  = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.2 CONFLICTOS", textFin= "5.3 CONFLICTOS")

# Reactivados
conflictosReactivados = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.3 CONFLICTOS", textFin= "VI. DETALLE DE LOS CONFLI")

# Latentes
conflictosLatentes = extraeTextoPdf(pdfPath = pdfPath, textIni = "VI. DETALLE DE LOS CONFLI", textFin= "6.1 CONFLICTOS")


In [6]:
encabezados = ["Reporte Mensual de Conflictos Sociales N.° 234 – Agosto 2023", "ADJUNTÍA PARA LA  PREVENCIÓN DE CONFLICTOS SOCIALES Y LA GOBERNABILIDAD – DEFENSORÍA DEL PUEBL O"]
for word in encabezados:
    conflictosActivos1 = conflictosActivos1.replace(word, "")

conflictosActivos1 = unidecode(conflictosActivos1)

In [7]:
departamentosPeru = [
    "Amazonas",
    "Ancash",
    "Apurimac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao",
    "Cusco",
    "Huancavelica",
    "Huanuco",
    "Ica",
    "Junin",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martin",
    "Tacna",
    "Tumbes",
    "Ucayali"
]

In [8]:
results = {}

# Identificar si los departamentos tienen conflictos
dptos = []
for dpto in departamentosPeru:
    try:
        encuentra = conflictosActivos1.index(dpto + "  ")
        dptos.append(dpto)
    except:
        continue

# Agrupación por dptos
nDpto = 0
for dpto in dptos:
    if dpto == dptos[-1]:
        inicio = conflictosActivos1.index(dptos[nDpto] + "  ")
        fin = len(conflictosActivos1)
        results[dpto] = {conflictosActivos1[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
    else:
        inicio = conflictosActivos1.index(dptos[nDpto] + "  ")
        fin = conflictosActivos1.index(dptos[nDpto + 1] + "  ")
        results[dpto] = {conflictosActivos1[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
        nDpto = nDpto + 1

    stopWords=["Tipo:","Ingreso como caso nuevo:", "Caso:","Ubicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NOHAYDIALOGO", "HAY DIALOGO"]
    textoDpto = list(results[dpto])[0]

    textoDpto = textoDpto.replace("Descripcion  Hechos del mes", "")
    textoDpto = textoDpto.replace("Hechos del mes  Hechos del mes", "")
    textoDpto = textoDpto.replace("CASO NUEVO", "")
    textoDpto = textoDpto.replace("Caso :", "Caso:")
    textoDpto = textoDpto.replace("Tipo :", "Tipo:")
    textoDpto = textoDpto.replace("Actores primarios :", "Actores primarios:")
    textoDpto = textoDpto.replace("Actore s primarios:", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores p rimarios:", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores pr imarios:", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores pri marios :", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores primari os:", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores primar ios:", "Actores primarios:")
    textoDpto = textoDpto.replace("Actores secundarios :", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actores  secundarios:", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actores secunda rios:", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actores secu ndarios:", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actor es secundarios:", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actores secundari os:", "Actores secundarios:")
    textoDpto = textoDpto.replace("Actores terciarios :", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actores Terciarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actores tercia rios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actores terc iarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actores te rciarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actor es terciarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actor es terciarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actor terciario :", "Actores terciarios:")
    textoDpto = textoDpto.replace("Actores t erciarios:", "Actores terciarios:")
    textoDpto = textoDpto.replace("Ubicacion :", "Ubicacion:")
    textoDpto = textoDpto.replace("Ubicaci on:", "Ubicacion:")
    textoDpto = textoDpto.replace("Ubicac ion:", "Ubicacion:")
    textoDpto = textoDpto.replace("Ingreso co mo caso nuevo:", "Ingreso como caso nuevo:")
    textoDpto = textoDpto.replace("Ingre so como caso nuevo:", "Ingreso como caso nuevo:")
    textoDpto = textoDpto.replace("Ingreso como caso nuevo :", "Ingreso como caso nuevo:")
    textoDpto = textoDpto.replace("Ingreso  como caso nuevo:", "Ingreso como caso nuevo:")
    textoDpto = textoDpto.replace("Ingreso como caso  nuevo:", "Ingreso como caso nuevo:")
    textoDpto = textoDpto.replace("NO HAY DIALOGO", "NOHAYDIALOGO")
    
    
    texto = textoDpto
    texto = texto.split("Tipo:")
    texto = texto[1:]
    texto = ["Tipo:" + elem for elem in texto]
    
    cont = 0
    for tex in texto:
        texConf = texto[cont]
        for word in stopWords:
            if word == "NOHAYDIALOGO" :
                texConf = texConf.replace(word, "|" + "Dialogo: NOHAYDIALOGO" + "|")
            elif word == "HAY DIALOGO":
                texConf = texConf.replace(word, "|" + "Dialogo: HAYDIALOGO" + "|")
            elif word == "Actores primarios:" or word == "Actores secundarios:" or word == "Actores terciarios:":
                texConf = texConf.replace(word, "|" + word)
            else:
                texConf = texConf.replace(word, "|" + word)
        texConf = texConf.split("|")
        texConf = texConf[1:]
        texto[cont] = texConf
        cont = cont +1
    
    results[dpto] = texto


resultsIncomp = {}

dptoList = []
tipoList = []
ingresoList = []
casoList = []
ubiList = []
actPrimList = []
actSecList = []
actTerList = []
dialogList = []
detalleList = []

for dpto in dptos:
    listConf = results[dpto]
    results[dpto] = [sorted(conf) for conf in listConf if len(conf) == 9]
    resultsIncomp[dpto] = [sorted(conf) for conf in listConf if len(conf) != 9]
    listConf = results[dpto]

    for conf in listConf:
        dptoList.append(dpto)
        detalleList.append(conf[0])
        actPrimList.append(conf[1])
        actSecList.append(conf[2])
        actTerList.append(conf[3])
        casoList.append(conf[4])
        dialogList.append(conf[5])
        ingresoList.append(conf[6])
        tipoList.append(conf[7])
        ubiList.append(conf[8])


listas = [dptoList, tipoList, ingresoList ,casoList, ubiList, actPrimList, actSecList, actTerList, dialogList, detalleList]
keyList = ["Dpto", "Tipo", "Ingreso como caso nuevo", "Caso", "Ubicacion","Actores primarios", "Actores secundarios", "Actores terciarios","Diálogo", "Detalle del mes"]
diccionario = {key: lista for key, lista in zip(keyList, listas)}
data = pd.DataFrame(diccionario)



In [56]:
#Conflicto en Arequipa no tiene actores secundarios --> print(results["Arequipa"][31])
#Conflicto en Cajamarca no tiene actores secundarios --> print(results["Cajamarca"][13])
#Conflicto en Cusco no tiene actores secundarios --> print(results["Cusco"][4])
#Conflicto en Cusco no tiene actores terciarios --> print(results["Cusco"][112])
#Conflicto en Lambayeque no tiene actores terciarios --> print(results["Lambayeque"][13])
#Conflicto en Moquegua no tiene actores terciarios --> print(results["Moquegua"][05])
#Conflicto en Piura no tiene actores terciarios --> print(results["Piura"][50])
#Conflicto en Puno no tiene actores terciarios --> print(results["Puno"][32])

Actores primarios: Coordinadora Regional de los Pueblos 
Indigenas de AIDESEP Atalaya (CORPIAA), Organizacion 
Indigena de la Region Atalaya  (OIRA), Federacion Asheninka 
de la Cuenca del Rio Unini (FACRU), Organizacion Indigena 
Yines de la Provincia de Atalaya (OIYPA), Federacion 
Ashaninka Bajo Urubamba (FABU), Federacion de 
Comunidades Nativas del Distrito de Sepahua (FECONADIS), 
Federacion d e Comunidades Nativas Asheninkas de la 
Provincia de Atalaya (FECONAPA),Federacion de Mujeres 
Indigenas de la Provincia de Atalaya (FEMIPA),Union Regional 
de Pueblos Indigenas de la Amazonia (URPIA),Confederacion 
